# Case 1: Implementing LS solver

by Ken Bastiaensen, Milan Van den Heuvel, Gonzalo Villa

*Advanced Econometrics 2016-2017. Case 1, least squares (LS) implementation.*

The system of linear equations is given in matrix notation
$$ y = X \beta + \mu$$

with dependent variables $y$ and residuals $\mu \in \mathbb{R}^{N\times1}$, independent variables $X \in \mathbb{R}^{N\times K}$ and coefficients $\beta \in \mathbb{R}^{K\times 1}$.

We use the estimator for the coefficients $\hat\beta = (X'X)^{-1}Xy$ as given in course. In practice more advanced methods are used for stability, see the bottom part of this document.

With $\hat\mu = y - X \hat\beta$, the standard error $\sigma$ is estimated by 
$$\hat\sigma^2 = \frac{\hat\mu'\hat\mu}{N - K}$$
and variance-covariance matrix of $\hat\beta$ given by
$$V[\hat\beta] = \hat\sigma^2 (X'X)^{-1}$$

The t distribution for this null hypothesis that $\beta_0=0$ is then
$$ t = \frac{\hat\beta - 0}{s.e.} $$ with the standard errors $s.e.$ on the diagonal of $\hat\sigma$.

In [1]:
#Load the Distributions package. Use `Pkg.install("Distributions")` to install first time.
using Distributions: TDist, cdf, ccdf

In [2]:
type ols_results
  coefs 
  yhat
  res
  vcv
  tstat
  pval
end

In [3]:
# Keyword arguments are placed after semicolon. 
# Symbols start with colon, e.g. `:symbol`.
function gls(y, X; corr=:none, lags=nothing)
    
    # more stable: β̂ = X \ y, see notes at bottom
    β̂ = inv(X' * X) * (X' * y)
    ŷ = X * β̂
    μ̂ = y - ŷ
    
    T, K = size(X)
    σ̂² = dot(μ̂, μ̂) / (T - K)

    #use correction for variance covariance
    if corr == :none
        vcv = σ̂² * inv(X' * X)
        
    elseif corr == :white
        # or do newey_west with lags=0
        vcv = inv(X' * X) * X' * diagm(μ̂.^2) * X * inv(X' * X)
        
    elseif corr == :newey_west
        if lags == nothing 
            lags = Int(floor(T^(1/4)))
        end
        vcv = newey_west(X, μ̂, lags)
    else
        error("wrong argument for correction keyword")
    end

    # T statistics for H₀: β₀ = 0
    tstat = β̂ ./ sqrt(diag(vcv))
    
    # absolute value and times two for double sided test
    pval  = 2 * ccdf(TDist(T-K), abs(tstat)) 

    return ols_results(β̂, ŷ, μ̂, vcv, tstat, pval)
end

gls (generic function with 1 method)

In [4]:
function newey_west(X, μ̂, lags::Integer)
    if lags == 0 #White estimator
        return inv(X'*X) * X' * diagm(μ̂.^2) * X * inv(X'*X)
    end
    
    T, k = size(X)
    vcv = zeros(k, k)
    for lag in 0:lags
        w = 1 - lag / (lags + 1)
        for t in (lag + 1):T
            # Calculates the off-diagonal terms
            update = w * μ̂[t] * μ̂[t-lag] * (X[t-lag,:]*X[t,:]' + X[t,:]*X[t-lag,:]')
            vcv = vcv + update
        end
    end  
    vcv = inv(X'*X) * vcv * inv(X'*X) 
end

newey_west (generic function with 1 method)

In [5]:
#test if it runs
gls(randn(20), randn(20,3))

ols_results([-0.79909,-0.281024,-0.659259],[0.89711,-0.445108,-0.873982,-0.583895,0.643354,-0.894633,-1.38627,-0.443149,0.684978,0.624576,0.99437,0.59485,-0.290817,-0.326096,0.396466,-0.861629,0.502337,0.273293,1.26224,-1.04805],[1.21303,-0.524702,-0.174645,1.81842,-0.236818,-0.0539331,0.734997,-0.35607,0.206723,-1.08922,-0.175208,2.33117,0.658781,1.44764,0.618347,1.70285,0.569568,-0.985123,1.02877,-0.428494],[0.110837 0.0218475 0.0228234; 0.0218475 0.0575132 0.0188665; 0.0228234 0.0188665 0.0688385],[-2.40023,-1.17182,-2.5127],[0.0281133,0.257433,0.0223588])

In [27]:
#simulation test
K = 3   # number of parameters
N = 100 # number of observations

# Create actual parameters and observations
β = randn(K) #[1, 10, 100]
X = randn(N, K)
X[:,1] = ones(N) # intercept
σ = 5
μ = σ * randn(N) # ~ N(0, σ)
y = X * β + μ;

In [28]:
res = gls(y, X)
@show β
@show res.coefs
res.vcv

β = [1.37092,0.22688,0.549384]
res.coefs = [1.63196,0.780497,1.63478]


3×3 Array{Float64,2}:
 0.254282    0.0229051   0.0252572
 0.0229051   0.28133    -0.0358594
 0.0252572  -0.0358594   0.256677 

In [29]:
res = gls(y, X, corr=:newey_west)
@show β
@show res.coefs
res.vcv

β = [1.37092,0.22688,0.549384]
res.coefs = [1.63196,0.780497,1.63478]


3×3 Array{Float64,2}:
  0.396468    -0.00738168  0.0763741
 -0.00738168   0.545238    0.020876 
  0.0763741    0.020876    0.592507 

In [9]:
# We test with the illustration of the slides 
y = [1673,1688,1666,1735,1749,1756,1815,1867,1948,2048,2128,2165,2257,2316,2324]
X2=[1839,1844,1831,1881,1883,1910,1969,2016,2126,2239,2336,2404,2487,2535,2595]
X = hcat(ones(length(X2)), X2, collect(1:15))
gls(y, X)

ols_results([300.286,0.741981,8.04356],[1672.83,1684.59,1682.98,1728.13,1737.65,1765.73,1817.55,1860.47,1950.13,2042.02,2122.03,2180.53,2250.16,2293.82,2346.38],[0.167431,3.41396,-16.9838,6.87355,11.346,-9.73102,-2.55145,6.53189,-2.12957,5.98303,5.96733,-15.5309,6.8411,22.1825,-22.38],[6133.65 -3.70794 220.206; -3.70794 0.00225946 -0.137052; 220.206 -0.137052 8.90154],[3.83421,15.6096,2.69597],[0.00237732,2.46242e-9,0.0194537])

# Estimate Cigarette Demand

In [10]:
data, header = readcsv("Data_Baltagi.csv", header=true)
println.(header);

state
 year
 P/pack
 pop
 pop16+
 CPI
 ndi/capita
 C/capita
 Pn/pack
 ln C_it
 ln P_it
 ln Pn_it
 ln Y_it


In [11]:
y = data[:, 10]
X = hcat(ones(y), data[:, 11:13])

1380×4 Array{Float64,2}:
 1.0  -0.0675933   -0.159065    3.93035
 1.0  -0.0394788   -0.119801    3.99498
 1.0  -0.0554792   -0.086146    4.05101
 1.0  -0.0281709   -0.0937682   4.0794 
 1.0  -0.0553988   -0.120782    4.10405
 1.0   0.0227283   -0.0838815   4.14772
 1.0  -0.00272851  -0.112348    4.17096
 1.0   0.0204089   -0.123275    4.20139
 1.0   0.0528969   -0.123354    4.23081
 1.0   0.0118907   -0.111226    4.28501
 1.0  -0.0531917   -0.174246    4.33463
 1.0  -0.134401    -0.174643    4.32325
 1.0  -0.143673    -0.224073    4.33054
 ⋮                                     
 1.0  -0.486196    -0.428838    4.70524
 1.0  -0.405983    -0.384444    4.65864
 1.0  -0.338482    -0.263871    4.59464
 1.0  -0.240375    -0.160426    4.60445
 1.0  -0.207925    -0.153377    4.6125 
 1.0  -0.113913    -0.0926677   4.61688
 1.0  -0.100871    -0.0673594   4.58327
 1.0  -0.0467213   -0.0256863   4.60289
 1.0  -0.0445251   -0.00809721  4.60509
 1.0  -0.00922374   0.0608496   4.66465
 1.0  -0.069937

In [12]:
res = gls(y, X)
@show res.coefs
@show res.pval;

res.coefs = [4.7478,-1.02434,0.259558,0.0655521]
res.pval = [7.77625e-243,1.88586e-61,8.03852e-6,0.00913432]


# Background on OLS
## Problem
Ordinary Least Squares (OLS) is a solution for a system of linear equations by minimizing the sum of squared differences between the observed values ($y$) and the corresponding modelled values ($\hat y = X \hat\beta$).

A system of equations is given by
$$ y_i = \beta_1 + \beta_2 X_{i2}  + \beta_3 X_{i3}  + ... + \beta_K X_{iK} + \mu_i,  \quad i=1,...,N$$

or in matrix notation
$$ y = X \beta + \mu$$

with $y, \mu \in \mathbb{R}^{N\times1}$, $X \in \mathbb{R}^{N\times K}$ and $\beta \in \mathbb{R}^{K\times 1}$.

The OLS solves
$$\operatorname{\,min} \, \big\|y - X \hat\beta \big\|^2$$
that with some algebra leads to the [normal equations](https://en.wikipedia.org/wiki/Normal_equations) 
$$ X' X\beta = X'y$$ 

## Solution

### Normal equations

In case of a [rank][1] complete matrix $X$ the normal equations can be solved by inverting the ([Gram](https://en.wikipedia.org/wiki/Gramian_matrix)) matrix $X'X$ such that $\hat\beta = (X'X)^{-1}Xy$.

As X'X is a symmetric, positive definite matrix it's computationally more efficient to use the cholesky decomposition of $X'X = LL'$ with $L$ a lower triangular matrix. This gives $LL'\hat\beta = X'y$. First solve $Lz = X'y$ for $z$ by [forward substitution](https://en.wikipedia.org/wiki/Forward_substitution) and then $L'\hat\beta = z$ for $\beta$ by backward substitution.

However, solving the normal equations is numerically unstable, i.e. it is very sensitive to small pertubations in $X$. The [condition number](https://en.wikipedia.org/wiki/Condition_number) $\kappa$ of the system is worsened: $\kappa(X'X) = [\kappa(X)]^2$. We show the impact further below.

[1]: https://en.wikipedia.org/wiki/Rank_(linear_algebra)

We first simulate some parameters and data using the same notation as in the slides.

In [13]:
K = 5   # number of parameters
N = 100 # number of observations

# Create actual parameters and observations
β = randn(K)
X = randn(N, K)
X[:,1] = ones(N) # intercept
σ = 0.1
μ = randn(N) * σ # ~ N(0,1)
y = X * β + μ;

In [14]:
#define function
OLS_normal(y, X) = inv(X'X) * X'y

# test and print result side by side for quick sanity check
β̂_normal = OLS_normal(y, X)
hcat(β̂_normal, β)

5×2 Array{Float64,2}:
  0.53793    0.544423
 -1.03249   -1.03373 
  0.287501   0.292956
  0.752885   0.74161 
 -1.347     -1.34078 

In [15]:
function OLS_chol(y, X)
    X′X_chol = cholfact(X'X)
    X′X_chol\(X'*y)
end

# test and print result side by side for quick sanity check
β̂_chol = OLS_chol(y, X)
hcat(β̂_chol, β)

5×2 Array{Float64,2}:
  0.53793    0.544423
 -1.03249   -1.03373 
  0.287501   0.292956
  0.752885   0.74161 
 -1.347     -1.34078 

### QR Factorization
We decompose $X$ to its orthogonal [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition):
$$X = Q\begin{bmatrix}
R \\
0
\end{bmatrix} $$
with $Q\in\mathbb{R}^{N\times N} $ an [orthogonal matrix](https://en.wikipedia.org/wiki/Orthogonal_matrix) and $R\in\mathbb{R}^{k\times k}$ an upper triangular matrix (with positive diagonal elements).
The solution is then given by
$$R \hat\beta =\left(Q' \mathbf y \right)_K$$

This is the default method in Julia for solving OLS (for non-square matrix $X$) with `β̂ = X\y`

In [16]:
function OLS_qr(y, X)
    X_qr = qrfact(X)
    X_qr \ y
end


OLS_qr (generic function with 1 method)

In [17]:
β̂_qr = OLS_qr(y, X)
hcat(β̂_qr, β)

5×2 Array{Float64,2}:
  0.53793    0.544423
 -1.03249   -1.03373 
  0.287501   0.292956
  0.752885   0.74161 
 -1.347     -1.34078 

### SVD
Another orthogonal decomposition uses the [singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) (SVD) of $X$:
$$X = U \Sigma V'$$
with $U \in\mathbb{R}^{N\times N}$ and $V \in\mathbb{R}^{K\times K}$ both an orthogonal matrix, and $\Sigma\in\mathbb{R}^{N\times K}$ a diagonal matrix. SVD can transform any matrix into a diagonal matrix with the right choice of orthogonal coordinate systems for its domain and range. This even works for rank deficient matrices and is numerically stable! In such an ill-conditioned system, SVD will return the solution $\hat\beta$ that has the smallest norm.




In [18]:
function OLS_svd(y, X)
    X_svd = svdfact(X)
    X_svd \ y
end

OLS_svd (generic function with 1 method)

In [19]:
β̂_svd = OLS_svd(y, X)
hcat(β̂_svd, β)

5×2 Array{Float64,2}:
  0.53793    0.544423
 -1.03249   -1.03373 
  0.287501   0.292956
  0.752885   0.74161 
 -1.347     -1.34078 

## Numerical Stability
We now test numerical stability (example 3.5 in [1]) with 
$$X = \begin{bmatrix}
1 & 1\\
1 & 1+\epsilon
\end{bmatrix}, 
y = \begin{bmatrix}
2 \\
2 
\end{bmatrix}$$
The solution is given by $$\hat\beta=\begin{bmatrix}2 \\ 0\end{bmatrix}$$ 

This system is ill-conditioned because a slight perturbation in $$y_e = \begin{bmatrix}
2 \\
2 + \epsilon
\end{bmatrix}$$
gives the compeltely different solution $$\hat\beta_e=\begin{bmatrix}1 \\ 1\end{bmatrix}$$

We show that methods using the normal equations do not find the second solution.

[1]: Numerically Efficient Methods for Solving Least Squares Problems, Do Q Lee.

In [20]:
ϵ = 1e-7
X = [1 1;1 1+ϵ]
y = [2; 2]
OLS_normal(y, X)

2-element Array{Float64,1}:
 2.0
 0.0

In [21]:
yₑ = [2; 2+ϵ]
@show OLS_normal(yₑ, X)
@show OLS_chol(yₑ, X);

OLS_normal(yₑ,X) = [0.75,1.125]
OLS_chol(yₑ,X) = [0.727273,1.27273]


In [22]:
@show OLS_qr(yₑ, X)
@show OLS_svd(yₑ, X);

OLS_qr(yₑ,X) = [1.0,1.0]
OLS_svd(yₑ,X) = [1.0,1.0]


### Advanced methods
Several advanced methods exist such as [bayesian OLS](https://en.wikipedia.org/wiki/Bayesian_linear_regression) (with possible priors), sparse OLS solvers, [James-Stein estimator](https://en.wikipedia.org/wiki/James%E2%80%93Stein_estimator) and [iterative solvers](https://en.wikipedia.org/wiki/Generalized_minimal_residual_method). 

In [23]:
using IterativeSolvers

In [24]:
OLS_lsmr(y, X) = lsmr(X, y)

OLS_lsmr (generic function with 1 method)

In [25]:
β̂_lsmr = OLS_lsmr(y, X)
hcat(β̂_lsmr, β)

LoadError: DimensionMismatch("vectors must have same lengths")